In [ ]:
!pip install requests pandas numpy tqdm
import time
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from tqdm import tqdm

def get_market_value(coin_id):
    url = f"https://api.coingecko.com/api/v3/coins/{coin_id}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        return data['market_data']['market_cap']['usd']
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for {coin_id}: {e}")
        return None

def collect_data(duration_hours=24, interval_minutes=5):
    bitcoin_data = []
    solana_data = []
    
    total_iterations = int(duration_hours * 60 / interval_minutes)
    
    with tqdm(total=total_iterations, desc="Collecting data", unit="interval") as pbar:
        for _ in range(total_iterations):
            bitcoin_mv = get_market_value('bitcoin')
            solana_mv = get_market_value('solana')
            
            timestamp = datetime.now()
            if bitcoin_mv is not None:
                bitcoin_data.append({'timestamp': timestamp, 'market_value': bitcoin_mv})
            if solana_mv is not None:
                solana_data.append({'timestamp': timestamp, 'market_value': solana_mv})
            
            pbar.update(1)
            
            time_left = timedelta(minutes=interval_minutes * (total_iterations - pbar.n))
            pbar.set_postfix_str(f"Time remaining: {time_left}")
            
            time.sleep(interval_minutes * 60)
    
    return pd.DataFrame(bitcoin_data), pd.DataFrame(solana_data)

def calculate_metrics(df):
    if df.empty:
        print("Not enough data to calculate metrics.")
        return df
    df['realized_value'] = df['market_value'].ewm(span=14).mean()
    df['mvrv'] = df['market_value'] / df['realized_value']
    df['mvrv_z_score'] = (df['mvrv'] - df['mvrv'].mean()) / df['mvrv'].std()
    return df

def main():
    print("Market Value Data is being collected. Please wait!")
    
    try:
        bitcoin_df, solana_df = collect_data()
        
        bitcoin_results = calculate_metrics(bitcoin_df)
        solana_results = calculate_metrics(solana_df)
        
        print("\n\nData collection and analysis complete.")
        
        if not bitcoin_df.empty:
            print("\nBitcoin Results:")
            print(bitcoin_results.tail())
            bitcoin_results.to_csv('bitcoin_results.csv', index=False)
            print("Bitcoin results have been saved to 'bitcoin_results.csv'")
        else:
            print("\nNo data collected for Bitcoin.")
        
        if not solana_df.empty:
            print("\nSolana Results:")
            print(solana_results.tail())
            solana_results.to_csv('solana_results.csv', index=False)
            print("Solana results have been saved to 'solana_results.csv'")
        else:
            print("\nNo data collected for Solana.")
        
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()

Market Value Data is being collected. Please wait!
